In [0]:
print("chart-studio")
!pip install chart-studio
print("gensim")
!pip install gensim
print("plotly")
!pip install plotly
print("pyLDAvis")
!pip install pyLDAvis
print("nltk")
!pip install nltk

In [0]:
import numpy as np
import pandas as pd
import re
import nltk

from plotly import version
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from chart_studio.grid_objs import Grid, Column
import plotly.graph_objs as go
import chart_studio.plotly as py
from plotly import tools
init_notebook_mode(connected = True)

import pyLDAvis
import pyLDAvis.gensim
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
pyLDAvis.enable_notebook()
from gensim import corpora,models

from scipy import stats

import warnings
warnings.filterwarnings('ignore')
import plotly.offline as pyo
pyo.init_notebook_mode()

In [0]:
!pwd
twitter_tweets = pd.read_csv('/content/sample_data/tweets.csv')
fake_users = pd.read_csv('/content/sample_data/users.csv')
tweets_details = pd.read_csv('/content/sample_data/tweetdetails.csv')

In [0]:
fake_users['Date'] = pd.to_datetime(fake_users['created_at'])
fake_users = fake_users[pd.notnull(fake_users['created_at'])]
fake_users = fake_users.drop_duplicates(subset=['id'])
fake_users['Date'] = fake_users['Date'].apply(lambda x: x.strftime('%Y-%m'))

In [0]:
u_name = pd.DataFrame(fake_users.name.str.split(' ',1).tolist(),columns = ['first','last'])
user_name = u_name.groupby('first',as_index=False).size().reset_index(name='counts')
users_name = user_name.sort_values('counts',ascending=False).head(20)


[['Robby', 'Delaware'], ['#Ezekiel2517✨...'], ['B', 'E C K S T E R✨'], ['Chris', 'Osborne'], ['Рамзан', 'Кадыров'], ['Маргарита', 'Саваж'], ['Dark', 'Nally'], ['Laila', 'Heats'], ['CC', 'Jack'], ['Mr', 'Moran'], ['Jonathon', 'Norwood'], ['Harper', 'H21'], ['Lara', 'Pretty'], ['Jasiah', 'Jenkins'], ['Jeremy', 'Roberts'], ['Reuben', 'Littlejohn'], ['Pauline', 'Gunn'], ['Ramona', 'Mcintosh'], ['Jeffrey', 'Kahunas'], ['Jason', 'Scott'], ['Sherman', 'Cannon'], ['Lovely', 'Cass'], ['Carrie'], ['Anthony', 'Nelson'], ['Wesley', 'Holmes'], ['Khloe', 'Irving'], ['Nick', 'Luna'], ['Messiah', 'Haynes'], ['Thomas', 'Lopez'], ['Cynthia', 'Mary Hunter'], ['Bossy', 'Bo$$'], ['Anton', 'Haynes'], ['Alec', 'Moody'], ['Jayda', 'Stone'], ['̧M̧ȩļa̧ņi̧ņ'], ['Nancy', 'CAMPBELL'], ['Jeanne', 'Mccarthy'], ['Jadon', 'Dailey'], ['Adrienne'], ['Laura', 'Baeley'], ['Bao', 'Ham'], ['Travi$', 'Lane'], ['Jared', 'Mallory'], ['Sarah', 'Richards'], ['Garret', 'Simpson'], ['Kate', 'Ritter'], ['Ernest', 'Rivers'], ['C

In [0]:
first_name = u_name.groupby('first',as_index=False).size().reset_index(name='counts')
first_name = first_name.sort_values('counts',ascending=False).head(20)
df = go.Bar(
    x=first_name['counts'],
    y=first_name['first'],
    orientation = 'h',
    name = 'First Name'
)

last_name = u_name.groupby('last',as_index=False).size().reset_index(name='counts')
last_name = last_name.sort_values('counts',ascending=False).head(20)
df1 = go.Bar(
    x=last_name['counts'],
    y=last_name['last'],
    orientation = 'h',
    name = 'Last Name'
)
fig = tools.make_subplots(rows=1,cols=2,subplot_titles=('First Name', 'Last Name'))
fig.append_trace(df,1,1)
fig.append_trace(df1,1,2)

fig['layout'].update(height=400,width=1000,title='first and Last Names of Fake Accounts')
plot(fig,filename = 'basic-line')

'basic-line.html'

In [0]:
des = fake_users.description.copy().astype(str)
des = des.str.replace('[^\w\s]','')
des = des.str.replace('[\\r|\\n|\\t|_]','')
des = des.str.strip()

tweets_des = tweets_details.copy()
tweets_des.des = des
tweets_des.des = tweets_des.des.apply(lambda x: ''.join([word for word in x.split() if word.lower() not in (stop)]))

In [0]:
doc_set = tweets_des.des.values.copy()
texts = [text.split(' ') for text in doc_set]
dictionary = corpora.Dictionary(texts)
corp = [dictionary.doc2bow(text) for text in texts]
ldamodel = models.ldamodel.LdaModel(corp, num_topics=30,id2word=dictionary)

In [0]:
df3 = pyLDAvis.gensim.prepare(ldamodel,corp,dictionary)
df3

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
8     -0.229474  0.004862       1        1  8.828117
18     0.075237 -0.159470       2        1  4.140618
13     0.079274  0.205698       3        1  4.140616
1     -0.023951 -0.001285       4        1  3.619791
10     0.073858 -0.008884       5        1  3.619789
26    -0.103501  0.002467       6        1  3.359379
25     0.071049 -0.005139       7        1  3.359375
5      0.071049 -0.005139       8        1  3.359374
12    -0.024635 -0.000503       9        1  3.359372
4      0.069613 -0.004089      10        1  3.229167
11    -0.157201 -0.009926      11        1  3.229167
19    -0.101492  0.002283      12        1  3.229166
27     0.069613 -0.004089      13        1  3.229162
29     0.069613 -0.004089      14        1  3.229161
22     0.068155 -0.003305      15        1  3.098961
24     0.068155 -0.003305      16        1  3.098961
28    -0.025034 -0.000053      17        1  3.098960
14     0.061176 -0.002077      18        1  3.098958
0     -0.025113  0.000110      19        1  2.968754
3     -0.025113  0.000110      20        1  2.968754
9      0.066673 -0.002692      21        1  2.968753
2      0.066673 -0.002692      22        1  2.968752
21    -0.025113  0.000110      23        1  2.968748
15    -0.096539  0.002004      24        1  2.838546
7     -0.025105  0.000250      25        1  2.838542
16     0.065165 -0.002197      26        1  2.838540
17     0.063630 -0.001782      27        1  2.708338
23     0.063630 -0.001783      28        1  2.708338
20    -0.129931  0.002408      29        1  2.447926
6     -0.110361  0.002197      30        1  2.447918, topic_info=    Category      Freq  ...  loglift  logprob
3    Default  28.00000  ...  30.0000  30.0000
144  Default   1.00000  ...  29.0000  29.0000
118  Default   1.00000  ...  28.0000  28.0000
269  Default   1.00000  ...  27.0000  27.0000
85   Default   1.00000  ...  26.0000  26.0000
280  Default   1.00000  ...  25.0000  25.0000
92   Default   1.00000  ...  24.0000  24.0000
14   Default   1.00000  ...  23.0000  23.0000
77   Default   1.00000  ...  22.0000  22.0000
160  Default   1.00000  ...  21.0000  21.0000
158  Default   1.00000  ...  20.0000  20.0000
94   Default   1.00000  ...  19.0000  19.0000
216  Default   1.00000  ...  18.0000  18.0000
66   Default   1.00000  ...  17.0000  17.0000
281  Default   1.00000  ...  16.0000  16.0000
58   Default   1.00000  ...  15.0000  15.0000
52   Default   1.00000  ...  14.0000  14.0000
7    Default   1.00000  ...  13.0000  13.0000
242  Default   1.00000  ...  12.0000  12.0000
231  Default   1.00000  ...  11.0000  11.0000
228  Default   1.00000  ...  10.0000  10.0000
108  Default   1.00000  ...   9.0000   9.0000
167  Default   1.00000  ...   8.0000   8.0000
279  Default   1.00000  ...   7.0000   7.0000
57   Default   1.00000  ...   6.0000   6.0000
159  Default   1.00000  ...   5.0000   5.0000
202  Default   1.00000  ...   4.0000   4.0000
204  Default   1.00000  ...   3.0000   3.0000
229  Default   1.00000  ...   2.0000   2.0000
38   Default   1.00000  ...   1.0000   1.0000
..       ...       ...  ...      ...      ...
56   Topic30   0.01471  ...  -0.4503  -6.4599
214  Topic30   0.01471  ...  -0.4503  -6.4599
13   Topic30   0.01471  ...  -0.4753  -6.4599
180  Topic30   0.01471  ...  -0.5025  -6.4599
88   Topic30   0.01471  ...  -0.5025  -6.4599
282  Topic30   0.01471  ...  -0.5025  -6.4599
146  Topic30   0.01471  ...  -0.5025  -6.4599
291  Topic30   0.01471  ...  -0.5025  -6.4599
114  Topic30   0.01471  ...  -0.9052  -6.4599
108  Topic30   0.01471  ...  -0.5698  -6.4599
109  Topic30   0.01471  ...  -0.5583  -6.4599
110  Topic30   0.01471  ...  -0.5641  -6.4599
111  Topic30   0.01471  ...  -0.5433  -6.4599
112  Topic30   0.01471  ...  -0.5641  -6.4599
113  Topic30   0.01471  ...  -0.5062  -6.4599
116  Topic30   0.01471  ...  -0.5433  -6.4599
115  Topic30   0.01471  ...  -0.5653  -6.4599
106  To

In [0]:
users = fake_users.groupby('Date', as_index = False).size().reset_index(name='counts')
userssum = users.sort_values('Date')

In [0]:
trace0 = go.Bar(name = "Accounts Created Over Time", x = userssum.Date, y = userssum.counts)
data = ([trace0])

layout=go.Layout(
    title = "accounts created 2009-2017",
    yaxis = dict(
        title = "No. of Accounts created",
        range = [0,100],
        titlefont=dict(
            size = 20,
        )
    ),
    
    xaxis = dict(
        title = 'Year',
        range = ['2009-01','2017-1'],
        titlefont=dict(
          size=20,
        )
    )
)
fig = go.Figure(data=data,layout=layout)
fig['layout'].update()
plot(fig)

'temp-plot.html'

In [0]:
m = pd.pivot_table(tweets_details,values = 'user_key',index = 'created_strDayofweek',columns='created_strMonth',aggfunc=len,fill_value=0,dropna=False)
z=m.as_matrix()

In [0]:
trace = go.Heatmap(z=z,
                   x=[i for i in np.arange(0,24)],
                   y=['Sunday','Monday','Tuseday','Wednesday','Thursday','Friday','Saterday'],
                   colorscale = 'Jet'
                  )
layout = go.Layout(
    title='No. of Tweets Per Day Per Month',
  xaxis=dict(
      nticks=24,
      title = 'month',
      titlefont=dict(
        size=20
      )
  ),
  yaxis = dict(
  ),  
)
data=[trace]
fig = go.Figure(data=data,layout=layout)
fig['layout'].update()
plot(fig, filename='tweets.html')

'tweets.html'

In [0]:
tweets_details = pd.read_csv('/content/sample_data/tweetdetails.csv',usecols={'user_key','retweet_count'})
tweets_details = tweets_details[pd.notnull(tweets_details['retweet_count'])]
tweets_details = tweets_details.groupby('user_key').sum().reset_index().sort_values('retweet_count',ascending=False).head(10)

print(tweets_details)
print('GoBar')
fig = go.Figure([
    go.Bar(x=tweets_details['user_key'],
    y=tweets_details['retweet_count'],
    orientation = 'v',
    name = 'retweet_count')
])
plot(fig,filename = 'basic-line')

           user_key  retweet_count
322   southlonestar        76435.0
346      tpartynews        71322.0
172     jenn_abrams        52995.0
129        gloed_up        51342.0
349   trayneshacole        44109.0
44   bleepthepolice        23535.0
170     jeblary2016        16691.0
294       redlanews        11676.0
299        rightnpr         8784.0
270     patriototus         8219.0
GoBar


'basic-line.html'